In [62]:
import numpy as np
import rasterio
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import os
import netCDF4
import h5netcdf
from mpl_toolkits.axes_grid1 import make_axes_locatable
import rioxarray as rio
import geopandas as gpd
import seaborn as sns

In [55]:
#load in temp data
temp_1999 = xr.open_dataset("../Temperature/Output Deliverable/export_1999.nc")
temp_2008 = xr.open_dataset("../Temperature/Output Deliverable/export_2008.nc")
temp_change = xr.open_dataset("../Temperature/Output Deliverable/export_change.nc")
#load in whale data
loc_1999 = pd.read_csv('exports/loc_1999.csv').drop('geometry', axis=1)
loc_2008 = pd.read_csv('exports/loc_2008.csv').drop('geometry', axis=1)
#turn to gpd
loc_1999 = gpd.GeoDataFrame(loc_1999, geometry=gpd.points_from_xy(x=loc_1999['long'], y=loc_1999['lat']))
loc_2008 = gpd.GeoDataFrame(loc_2008, geometry=gpd.points_from_xy(x=loc_2008['long'], y=loc_2008['lat']))

,time,long,lat,id,distances (m),geometry
0,2008-01-02 00:06:49,-115.200,27.641,2007CA-Bmu-00831,0.000000,POINT (-115.20000 27.64100)
1,2008-01-02 00:54:18,-115.135,27.690,2007CA-Bmu-00831,8419.392299,POINT (-115.13500 27.69000)
2,2008-01-02 12:25:27,-115.023,27.512,2007CA-Bmu-00831,22482.824451,POINT (-115.02300 27.51200)
3,2008-01-04 00:29:03,-114.749,27.234,2007CA-Bmu-00831,40976.595090,POINT (-114.74900 27.23400)
4,2008-01-06 00:08:14,-114.615,27.270,2007CA-Bmu-00831,13997.822229,POINT (-114.61500 27.27000)
...,...,...,...,...,...,...
3311,2008-06-03 19:13:04,-115.027,26.702,2008CR-Bmu-01388,239434.408767,POINT (-115.02700 26.70200)
3312,2008-06-04 22:27:15,-115.930,26.972,2008CR-Bmu-01388,95617.097297,POINT (-115.93000 26.97200)
3313,2008-06-11 11:42:10,-119.121,30.734,2008CR-Bmu-01388,519095.051717,POINT (-119.12100 30.73400)
3314,2008-06-11 11:59:00,-119.172,30.742,2008CR-Bmu-01388,4984.648120,POINT (-119.17200 30.74200)
